# Models for Project 4

### Introduction
In this notebook, I hope to answer the following questions:

1. Are wine points predictable based on it's chracteristics?

##### Step 1 Imports and Clean-up

Importing data and packages that will be used for analysis

In [87]:
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import stats, mannwhitneyu
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn import metrics, linear_model, model_selection
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#from sklearn.naive_bayes import MultinomialNB    
from sklearn.metrics import r2_score
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_selection import SelectFromModel
#importing unicodedata to handle é in rosé wine variety
import unicodedata
%matplotlib inline 

Load dataset and save as 'wine' variable.
After importing all dependencies we load our dataset called wine.

In [2]:
wine = pd.read_json('./wine-reviews/winemag-data-130k-v2.json', dtype='unicode');

Cleaning up accent marks within dataset by replacing Rosé with Rose.

In [3]:
wine.loc[:, 'variety'].replace(u'Rosé','Rose', inplace=True)

Checking for null values and reveiwing data information.

In [4]:
wine.isnull().sum()

country                  0
description              0
designation              0
points                   0
price                    0
province                 0
region_1                 0
region_2                 0
taster_name              0
taster_twitter_handle    0
title                    0
variety                  0
winery                   0
dtype: int64

In [5]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 13 columns):
country                  129971 non-null object
description              129971 non-null object
designation              129971 non-null object
points                   129971 non-null object
price                    129971 non-null object
province                 129971 non-null object
region_1                 129971 non-null object
region_2                 129971 non-null object
taster_name              129971 non-null object
taster_twitter_handle    129971 non-null object
title                    129971 non-null object
variety                  129971 non-null object
winery                   129971 non-null object
dtypes: object(13)
memory usage: 12.9+ MB


Changing points and price types to integar and float

In [6]:
#changing dtypes for points and price
wine['points'] = wine.points.astype(int)
wine['price'] = wine.price.astype(float)
wine.head(2)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,None,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,None,None,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


In [7]:
#Since the wine price is an object with 'nan' text values 
#I needed to find a way to remove the values without using dropna.
wine.dropna(inplace=True)

##### Step 2 Dummy Data
The next step to building our model is to dummy the data. I've dummied country, designation, province, region_1, region_2, and variety. This will allow us to assign numeric values to categories which we'll use in our model.

In [8]:
dummy = pd.get_dummies(wine[['country']],prefix='dum_country', drop_first=True)
print(dummy.columns)

Index(['dum_country_Armenia', 'dum_country_Australia', 'dum_country_Austria',
       'dum_country_Bosnia and Herzegovina', 'dum_country_Brazil',
       'dum_country_Bulgaria', 'dum_country_Canada', 'dum_country_Chile',
       'dum_country_China', 'dum_country_Croatia', 'dum_country_Cyprus',
       'dum_country_Czech Republic', 'dum_country_England',
       'dum_country_France', 'dum_country_Georgia', 'dum_country_Germany',
       'dum_country_Greece', 'dum_country_Hungary', 'dum_country_India',
       'dum_country_Israel', 'dum_country_Italy', 'dum_country_Lebanon',
       'dum_country_Luxembourg', 'dum_country_Macedonia', 'dum_country_Mexico',
       'dum_country_Moldova', 'dum_country_Morocco', 'dum_country_New Zealand',
       'dum_country_None', 'dum_country_Peru', 'dum_country_Portugal',
       'dum_country_Romania', 'dum_country_Serbia', 'dum_country_Slovakia',
       'dum_country_Slovenia', 'dum_country_South Africa', 'dum_country_Spain',
       'dum_country_Switzerland', 'dum_c

In [9]:
dummy = pd.get_dummies(wine[['designation']],prefix='dum_designation', drop_first=True)
print(dummy.columns)

Index(['dum_designation_#50 Mon Chou', 'dum_designation_#SocialSecret',
       'dum_designation_%@#$!', 'dum_designation_&',
       'dum_designation_'61 Rosé', 'dum_designation_'A Rina',
       'dum_designation_'Blend 105' Red Wine', 'dum_designation_'Na Vota',
       'dum_designation_'S'', 'dum_designation_'Unfiltered'',
       ...
       'dum_designation_Ürziger Würzgarten GG Réserve Alte Reben Erste Lage Dry',
       'dum_designation_Ürziger Würzgarten Kabinett',
       'dum_designation_Ürziger Würzgarten Spätlese',
       'dum_designation_étoile Brut', 'dum_designation_étoile Rosé',
       'dum_designation_Župska', 'dum_designation_‘Rough Justice' Red',
       'dum_designation_‘S'', 'dum_designation_“Champ” Jim the Gent',
       'dum_designation_“Champ” Lightnin' Lane'],
      dtype='object', length=35776)


In [10]:
dummy = pd.get_dummies(wine[['province']],prefix='dum_province', drop_first=True)
print(dummy.columns)

Index(['dum_province_Aconcagua Costa', 'dum_province_Aconcagua Valley',
       'dum_province_Aegean', 'dum_province_Agioritikos', 'dum_province_Ahr',
       'dum_province_Alenquer', 'dum_province_Alentejano',
       'dum_province_Alentejo', 'dum_province_Alenteo', 'dum_province_Algarve',
       ...
       'dum_province_Wellington', 'dum_province_Western Australia',
       'dum_province_Western Cape', 'dum_province_Wiener Gemischter Satz',
       'dum_province_Württemberg', 'dum_province_Zenata',
       'dum_province_Österreichischer Perlwein',
       'dum_province_Österreichischer Sekt', 'dum_province_Štajerska',
       'dum_province_Župa'],
      dtype='object', length=422)


In [11]:
dummy = pd.get_dummies(wine[['region_1']],prefix='dum_region_1', drop_first=True)
print(dummy.columns)

Index(['dum_region_1_Adelaida District', 'dum_region_1_Adelaide',
       'dum_region_1_Adelaide Hills', 'dum_region_1_Adelaide Plains',
       'dum_region_1_Aglianico d'Irpinia',
       'dum_region_1_Aglianico del Beneventano',
       'dum_region_1_Aglianico del Taburno',
       'dum_region_1_Aglianico del Vulture', 'dum_region_1_Agrelo',
       'dum_region_1_Albana di Romagna',
       ...
       'dum_region_1_Yadkin Valley', 'dum_region_1_Yakima Valley',
       'dum_region_1_Yamhill County', 'dum_region_1_Yarra Valley',
       'dum_region_1_Yecla', 'dum_region_1_Yolo County',
       'dum_region_1_York Mountain', 'dum_region_1_Yorkville Highlands',
       'dum_region_1_Yountville', 'dum_region_1_Zonda Valley'],
      dtype='object', length=1204)


In [12]:
dummy = pd.get_dummies(wine[['region_2']],prefix='dum_region_2', drop_first=True)
print(dummy.columns)

Index(['dum_region_2_Central Coast', 'dum_region_2_Central Valley',
       'dum_region_2_Columbia Valley', 'dum_region_2_Finger Lakes',
       'dum_region_2_Long Island', 'dum_region_2_Napa',
       'dum_region_2_Napa-Sonoma', 'dum_region_2_New York Other',
       'dum_region_2_None', 'dum_region_2_North Coast',
       'dum_region_2_Oregon Other', 'dum_region_2_Sierra Foothills',
       'dum_region_2_Sonoma', 'dum_region_2_South Coast',
       'dum_region_2_Southern Oregon', 'dum_region_2_Washington Other',
       'dum_region_2_Willamette Valley'],
      dtype='object')


In [13]:
dummy = pd.get_dummies(wine[['variety']],prefix='dum_variety', drop_first=True)
print(dummy.columns)
dummy.shape

Index(['dum_variety_Agiorgitiko', 'dum_variety_Aglianico',
       'dum_variety_Aidani', 'dum_variety_Airen', 'dum_variety_Albana',
       'dum_variety_Albanello', 'dum_variety_Albariño',
       'dum_variety_Albarossa', 'dum_variety_Aleatico',
       'dum_variety_Alfrocheiro',
       ...
       'dum_variety_Xynisteri', 'dum_variety_Yapincak', 'dum_variety_Zibibbo',
       'dum_variety_Zierfandler', 'dum_variety_Zierfandler-Rotgipfler',
       'dum_variety_Zinfandel', 'dum_variety_Zlahtina', 'dum_variety_Zweigelt',
       'dum_variety_Çalkarası', 'dum_variety_Žilavka'],
      dtype='object', length=697)


(120975, 697)

In [14]:
# Concatenate the original DataFrame and the dummy DataFrame (axis=0 means rows, axis=1 means columns).
dummy_wine = pd.concat([wine, dummy], axis=1)

##### Step 3: Feature Selection
The next step before we start to consider modeling our data is to select all the features or all of my 'X' variables. This process will help us determine which features have the greatest impact on predicting our outcome or 'y' variable. 

The code below is designed to find the correlation of points to each wine feature.

In [15]:
#save the correlation function in a variable
corr_dummy_wine = dummy_wine.corr()  

In [16]:
corr_points = corr_dummy_wine.loc[:, ['points']]
# find the 15 most pos correlated features to star rating 
print(corr_points.nlargest(25, 'points'))             
print('------------------------------')
# find the 15 most neg correlated features to star rating
print(corr_points.nsmallest(25, 'points'))             

                                        points
points                                1.000000
price                                 0.416167
dum_variety_Pinot Noir                0.111451
dum_variety_Nebbiolo                  0.087904
dum_variety_Riesling                  0.069101
dum_variety_Sangiovese Grosso         0.054668
dum_variety_Syrah                     0.053358
dum_variety_Grüner Veltliner          0.051174
dum_variety_Champagne Blend           0.040423
dum_variety_Port                      0.033572
dum_variety_Bordeaux-style Red Blend  0.026134
dum_variety_Rhône-style Red Blend     0.025424
dum_variety_Blaufränkisch             0.022330
dum_variety_Portuguese Red            0.019759
dum_variety_Cabernet Sauvignon        0.017977
dum_variety_Shiraz                    0.016929
dum_variety_Aglianico                 0.015824
dum_variety_Austrian white blend      0.015097
dum_variety_Tinta de Toro             0.014080
dum_variety_Sagrantino                0.013986
dum_variety_T

##### Can we predict the points rating based on other characteristics of a wine? 

##### Linear Regression
Using linear regression I wanted to try to predict the outcome variable (in this case rating points) based on all X variables (or column features including dummies). Before we run the model we split the data into a train/test split. This allows us train our model so we can use it on other data later. 

How big is the dummy wine dataset?

In [17]:
dummy_wine.shape

(120975, 710)

Null Hypothesis Regression

In [89]:
y_null = np.full((len(dummy_wine.points), ), dummy_wine.points.mean())
print("RMSE",np.sqrt(metrics.mean_squared_error(y_null, y_wine)))
print("R-Squared",metrics.r2_score(y_null, y_wine))

#(scale -1 to 1) High R-squared means more accurate. Low is inaccurate. 
#My prediction for lin reg can not explain the variation of y by my x

RMSE 3.0444953794531355
R-Squared -1.1474406946010606e+28


Linear Regression with price feature only

In [68]:
features1 = ['price']

X_wine = wine[features1]                # used to make prediction                                                    
y_wine = wine['points']                # what we want to predict 

X_train_wine, X_test_wine, y_train_wine, y_test_wine = train_test_split(X_wine, y_wine, random_state = 413)

# train data is what the model is fed
# testing data is new and shows how well the model can adjust
linreg = LinearRegression()
linreg.fit(X_train_wine, y_train_wine)
    
y_pred_wine = linreg.predict(X_test_wine)

print("Points value if all X variables were ZERO",linreg.intercept_) # print the interecept and the slope coeffecient
print("R-sq train", linreg.score(X_train_wine,y_train_wine)) #output the score or the rsqared of the model
print("R-sq test", linreg.score(X_test_wine,y_test_wine)) #output the score or the rsqared of the model

print("RMSE",np.sqrt(metrics.mean_squared_error(y_test_wine, y_pred_wine)))  #spit out RMSE

list(zip(features, linreg.coef_))

Points value if all X variables were ZERO 87.37127353791841
R-sq train 0.16754316064753183
R-sq test 0.1890867277908327
RMSE 2.7448290922077474


[('price', 0.029706128437876737)]

R-squared is measurement of how accurate the model is predicting our outcome. We want the result to be closer to 1 as it's a measurement for greater accuracy. With the model above, using price as the only feature to predict a points rating, our R-squared is 0.17 on our training set and 0.19 on our test set. This is relatively low and we can determing that this wouldn't be a reliable model.

Linear Regression with all column features

In [76]:
#linear regression with dummy features
features2 = dummy_wine.drop(columns=['country','province','region_1','region_2','variety','points', 'description', 'designation','taster_name', 'taster_twitter_handle','title','winery']) 
#for dummies maybe use only top 10 countries, 20 regions etc -- drop ones I don't need to use in features

# used to make prediction
X_dummy_wine2 = dummy_wine[features2.columns]  

# what we want to predict
y_dummy_wine2 = dummy_wine.points                

#split train test
X_train_dummy_wine2, X_test_dummy_wine2, y_train_dummy_wine2, y_test_dummy_wine2 = train_test_split(X_dummy_wine2, y_dummy_wine2, random_state = 413)

# train data is what the model is fed
# testing data is new and shows how well the model can adjust
linreg = LinearRegression()
linreg.fit(X_train_dummy_wine2, y_train_dummy_wine2)
    
y_pred_dummy_wine2 = linreg.predict(X_test_dummy_wine2)

print("Points value if all X variables were ZERO",linreg.intercept_) # print the interecept and the slope coeffecient
print("R-sq train", linreg.score(X_train_dummy_wine2,y_train_dummy_wine2)) #output the score or the rsqared of the model
print("R-sq test", linreg.score(X_test_dummy_wine2,y_test_dummy_wine2)) #output the score or the rsqared of the model
print("RMSE",np.sqrt(metrics.mean_squared_error(y_test_dummy_wine2, y_pred_dummy_wine2)))  #spit out RMSE

list(zip(features, linreg.coef_))

Points value if all X variables were ZERO 85.31679196828883
R-sq train 0.4138121535441422
R-sq test -2.6299125338966788e+16
RMSE 494309271.39677715


[('price', 0.015182232944445597),
 ('dum_variety_Agiorgitiko', 1.717280312200348),
 ('dum_variety_Aglianico', 2.7277973887659823),
 ('dum_variety_Aidani', -3.7266934446117457),
 ('dum_variety_Airen', -5.453336179365511),
 ('dum_variety_Albana', 2.827064072071014),
 ('dum_variety_Albanello', 0.3794928292984221),
 ('dum_variety_Albariño', 2.130493955753462),
 ('dum_variety_Albarossa', -5191954840.108746),
 ('dum_variety_Aleatico', -0.17028583213686943),
 ('dum_variety_Alfrocheiro', 3.0194926159456372),
 ('dum_variety_Alicante', 2.0877444008365273),
 ('dum_variety_Alicante Bouschet', 3.051955981180072),
 ('dum_variety_Aligoté', 0.7009198367595673),
 ('dum_variety_Alsace white blend', 3.2185350209474564),
 ('dum_variety_Altesse', 3.4120823070406914),
 ('dum_variety_Alvarelhão', 0.8644246710464358),
 ('dum_variety_Alvarinho', 2.9397341907024384),
 ('dum_variety_Alvarinho-Chardonnay', 1.531363107264042),
 ('dum_variety_Ansonica', 0.22772811772301793),
 ('dum_variety_Antão Vaz', 2.23039594292

We're running another linear regression here but instead of using only price as our only feature or 'X' value we're using all non-numeric columns (which include dummy columns) to predict our 'y' or points value. As you can see we still have low R-square values which isn't a good indication of a reliable dataset.

Linear Regression with price and variety dummy features

In [62]:
#linear regression with variety dummy and price features
features3 = dummy_wine.filter(regex='dum_variety(.*)|(price)')
#for dummies maybe use only top 10 countries, 20 regions etc -- drop ones I don't need to use in features

# used to make prediction
X_dummy_var = dummy_wine[features3.columns]  

# what we want to predict
y_dummy_var = dummy_wine.points                

#split train test
X_train_dummy_var, X_test_dummy_var, y_train_dummy_var, y_test_dummy_var = train_test_split(X_dummy_var, y_dummy_var, random_state = 413)

# train data is what the model is fed
# testing data is new and shows how well the model can adjust
linreg = LinearRegression()
linreg.fit(X_train_dummy_var, y_train_dummy_var)
    
y_pred_dummy_var = linreg.predict(X_test_dummy_var)

print("Points value if all X variables were ZERO",linreg.intercept_) # print the interecept and the slope coeffecient
print("R-sq train", linreg.score(X_train_dummy_var,y_train_dummy_var)) #output the score or the rsqared of the model
print("R-sq test", linreg.score(X_test_dummy_var,y_test_dummy_var)) #output the score or the rsqared of the model
print("RMSE",np.sqrt(metrics.mean_squared_error(y_test_dummy_var, y_pred_dummy_var)))  #spit out RMSE

list(zip(features, linreg.coef_))

Points value if all X variables were ZERO 84.7726248267295
R-sq train 0.2270818925842809
R-sq test 0.23716428845750848
RMSE 2.6622179775641177


[('price', 0.027275003850463622),
 ('dum_variety_Agiorgitiko', 1.9794556957765956),
 ('dum_variety_Aglianico', 3.51186212970117),
 ('dum_variety_Aidani', -3.509049930692198),
 ('dum_variety_Airen', -5.018099861383481),
 ('dum_variety_Albana', 3.730737594336311),
 ('dum_variety_Albanello', 0.6818750962610705),
 ('dum_variety_Albariño', 2.434016938834217),
 ('dum_variety_Albarossa', 6.394884621840902e-14),
 ('dum_variety_Aleatico', -0.10680622978548024),
 ('dum_variety_Alfrocheiro', 3.5839519837953486),
 ('dum_variety_Alicante', 2.3568688356722887),
 ('dum_variety_Alicante Bouschet', 3.74963718818183),
 ('dum_variety_Aligoté', 1.0077132555744228),
 ('dum_variety_Alsace white blend', 4.865181995997813),
 ('dum_variety_Altesse', 3.740321533083804),
 ('dum_variety_Alvarelhão', 1.1546000924232491),
 ('dum_variety_Alvarinho', 3.3173966138887008),
 ('dum_variety_Alvarinho-Chardonnay', 1.954625134770715),
 ('dum_variety_Ansonica', 0.40912505775657515),
 ('dum_variety_Antão Vaz', 2.5364212512918

We're running one more linear regression model but instead of using all columns we're only using variety of wine and price to determine the points value. Again, with low R-squared values we can confidently say that linear regression isn't a good model to predict the points rating.

#### Logistic Regression
Logisitic Regression modeling basically outputs the probabilities of a specific class which can be used to predict classification. This model isn't ideal for predicting the actual point value, however, we can use it to determine if wine is good or bad. We do this by setting a threshold for good wine which equals any wine rated over 88 ( which is the mean ). With that threshold we can determine the probability that a wine is good or bad. 

Note: 

Since all wines have a good rating from 80 to 100, assuming we're rating the wine on a scale of 1 to 100, we will need to determine the scale of a good or bad wine. For example, if a wine has a rating of 87 is it considered good or bad due to our 80 to 100 scale? 

Creating class variable for logisitic regression model

In [42]:
dummy_wine['good_bad_rating'] = dummy_wine.points > 92
#anything above the mean/median has good point rating otherwise bad

What is the null value for logisitic regression/classification?

In [86]:
dummy_wine.good_bad_rating.value_counts(normalize=True)

False    0.904154
True     0.095846
Name: good_bad_rating, dtype: float64

In [43]:
#K Fold 
kf = model_selection.KFold(n_splits=5, shuffle=True)
#Log Reg init
log = LogisticRegression(C=1e9, multi_class='multinomial', solver='lbfgs')

#logistic regression with dummy variety and price features. 
#We determine the best features were variety and price due to the coefficient strength
features_log = dummy_wine.filter(regex='dum_variety(.*)|(price)')

#Features
A = dummy_wine[features_log.columns]

#y
dummy_wine['gb_rate'] = dummy_wine.good_bad_rating.map({True:1, False:0})
b = dummy_wine['gb_rate']
#Split train test
A_train, A_test, b_train, b_test = train_test_split(A,b, random_state=413, test_size=0.33)

#fitting our model  
log.fit(A_train,b_train)
#prediction
b_pred = log.predict(A_test)
#scores
print('Accuracy Score: ')
print(metrics.accuracy_score(b_test,b_pred))
print('Confusion Matrix: ')
print(metrics.confusion_matrix(b_test,b_pred))
print('Cross Val Score: ')
print(cross_val_score(log, A, b, cv=kf, scoring='accuracy').mean())
print('---')
print("Coefficient:") #weights of each feature
print(list(zip(features_log, log.coef_[0])))
print("Intercept:") #value of intercept
print(log.intercept_)
print("Predict:")
print(log.predict_proba(A_test))

Accuracy Score: 
0.9120535043334502
Confusion Matrix: 
[[35745   402]
 [ 3109   666]]
Cross Val Score: 
0.9116842322793965
---
Coefficient:
[('price', 0.01857855341495584), ('dum_variety_Agiorgitiko', -0.03199104398655296), ('dum_variety_Aglianico', 0.12095266640534799), ('dum_variety_Aidani', -0.0007423591272113818), ('dum_variety_Airen', -0.0004134834687565452), ('dum_variety_Albana', 0.020471234844934176), ('dum_variety_Albanello', -0.000588361534938573), ('dum_variety_Albariño', -0.17269991510536353), ('dum_variety_Albarossa', 0.0), ('dum_variety_Aleatico', -0.0029777350799871393), ('dum_variety_Alfrocheiro', -0.008382430007693524), ('dum_variety_Alicante', -0.0026583015136798067), ('dum_variety_Alicante Bouschet', -0.005891355019409911), ('dum_variety_Aligoté', -0.01068635661434893), ('dum_variety_Alsace white blend', 0.10644378506882934), ('dum_variety_Altesse', -0.0038348732513304663), ('dum_variety_Alvarelhão', -0.0012221999349709), ('dum_variety_Alvarinho', -0.0422409698467201

After we run our model we get an accuracy score of 91 which is the percentage of correct predictions.

In [24]:
pd.DataFrame({'Features':features_log.columns, 'Coefs':log.coef_[0]}).sort_values(by='Coefs')

,Features,Coefs
486,dum_variety_Rose,-0.551344
682,dum_variety_White Blend,-0.345968
430,dum_variety_Pinot Grigio,-0.291027
578,dum_variety_Tempranillo,-0.290088
322,dum_variety_Merlot,-0.280735
69,dum_variety_Cabernet Franc,-0.246954
41,dum_variety_Barbera,-0.243144
152,"dum_variety_Corvina, Rondinella, Molinara",-0.234933
104,dum_variety_Carmenère,-0.234218
119,dum_variety_Champagne Blend,-0.217948


#### Random Forests

Using the same variables as the logistic regression above, let's use random forest to determine feature importance. A Random Forest Regression Algorithm predicts the outcome based on a series of Decision Trees.  


In [25]:
from sklearn.ensemble import RandomForestRegressor

In [26]:
# max_features=5 is best and n_estimators=150 is sufficiently large.
rfreg = RandomForestRegressor(n_estimators=150,
                              max_features=5,
                              oob_score=True,
                              n_jobs = -1,
                              random_state=1)

In [27]:
# Define X and y.
A2 = dummy_wine.description
b2 = dummy_wine.points

In [28]:
# Split the new DataFrame into training and testing sets.
A2_train, A2_test, b2_train, b2_test = train_test_split\
(A2, b2, random_state=413, test_size=0.3)


# Use CountVectorizer to create document-term matrices from X_train and X_test.
vtr = CountVectorizer(lowercase=True, ngram_range=(1, 2))
A2_train_nlp = vtr.fit_transform(A2_train)
A2_test_nlp = vtr.transform(A2_test)

In [29]:
# Create object that selects features with importance greater than or equal to a threshold
selector = SelectFromModel(rfreg, threshold='mean')

In [30]:
# Feature new feature matrix using selector
A2_train_important = selector.fit_transform(A2_train_nlp, b2_train)
A2_test_important = selector.fit_transform(A2_test_nlp, b2_test)

In [32]:
# Fit the model on only the train data
rfreg.fit(A2_train_important, b2_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=150, n_jobs=-1, oob_score=True, random_state=1,
           verbose=0, warm_start=False)

In [33]:
#Check the RMSE for a random forest that only 
#includes important features.
scores_test = cross_val_score(rfreg, A2_test_important, b2_test, cv=5, scoring='neg_mean_squared_error')
print('RMSE Test: ',np.mean(np.sqrt(-scores_test)))
print('out of bag: ',(rfreg.oob_score_))            


#b2_pred = rfreg.predict(A2_test_important) 
#print("R-sq train", rfreg.score(A2_nlp,b2))
#print("R-sq train", np.sqrt(metrics.mean_squared_error(b2_test, b2_pred)))

RMSE Test:  2.4051346083864837
out of bag:  0.4306263036272373


In [34]:
# Check the RMSE for a random forest that only includes important features.
scores_train = cross_val_score(rfreg, A2_train_important, b2_train, cv=5, scoring='neg_mean_squared_error')
print('RMSE Train: ',np.mean(np.sqrt(-scores_train)))
print('out of bag: ',(rfreg.oob_score_))         

RMSE Train:  2.308128679857256
out of bag:  0.4306263036272373


In [35]:
A2_nlp = vtr.fit_transform(A2)

# Find the average RMSE.
kf = KFold(n_splits=3, shuffle=True)
scores = cross_val_score(rfreg, A2_nlp, b2, cv=kf, scoring='neg_mean_squared_error')
print(np.mean(np.sqrt(-scores)))

2.357074400617728


In [36]:
pd.DataFrame({'Features':features_log.columns, 'Coefs':rfreg.feature_importances_[0]}).sort_values(by='Coefs')

,Features,Coefs
0,price,0.000033
460,dum_variety_Prugnolo Gentile,0.000033
461,dum_variety_Prunelard,0.000033
462,dum_variety_Pugnitello,0.000033
463,dum_variety_Rabigato,0.000033
464,dum_variety_Raboso,0.000033
465,dum_variety_Ramisco,0.000033
466,dum_variety_Rara Neagra,0.000033
467,dum_variety_Rebo,0.000033
468,dum_variety_Rebula,0.000033


In terms of trying to predict the actual points value linear regression isn't the best model to do that with this dataset. With a RMSE of 2.74 (when using price only as 'X') it's safe to say that this model isn't reliable. 

That said, Random Forest Regression did slightly better when trying to predict points with a RMSE of 2.42. While this isn't enough to use for a real world problem, it is our best model when trying to predict the point rating value. 

The other model I ran was a logistic regression model to solve for rating a wine good or bad. I classified wines over a 90 rating as being a good wine, which is 2 points higher than the mean and one point less than top 75% percentile. The model's accuracy score is 70 which is the percentage of correct predictions.

Feature importance for each model didn't changed too much from model to model. Each model listed the most important characteristics of wine, according to this dataset, as price and the various varieties of wine.

Note: Tried tuning n_estimators, however, the tax on my computing resources was too much. The function was taking more than 4 hours and still hadn't finished.